#Lab.11 / IBM3202 – Prediction of interactions from the coevolutionary analysis of sequence information

###Theoretical aspects

As we have discussed, **natural protein sequences can vary highly** in terms of sequence identity and yet **retain their structure** and function. A striking example of this is the **AAA protein family of ATPases** (Fig. 1), present in all domains in life. All family members have a common AAA-cassette responsible for nucleotide binding and hydrolysis although **displaying sequence identity as low as 20%**. These results show that structure and function conservation can exist even in poorly conserved sequences. The same observations can be made for the analysis of **non-coding RNAs** that catalyze biochemical reactions and bind to specific ligands by **folding into complex three-dimensional structures**, such as in the case of ribozymes and riboswitches.


<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/dca_01.png'/>
<figcaption>FIGURE 1. Architecture and sequence conservation of the AAA cassette, with its different domains shown as colored boxes, illustrating the low sequence identity of these proteins. Sequence patterns typical of the AAA family are listed in detail. In addition to the standard one-letter code for amino acids, the following symbols are used: <b>-</b>= D/E, <b>+</b>= R/K/H, <b>*</b>= charged , <b>O</b> = large and hydrophobic. Sequence colors denote conservation: red, green, black means almost absolutely conserved, well-conserved, and conserved, respectively. <br> Beyer A (1997) <i> Protein Sci 6(10), 2043-2058</i></figcaption></center>
</figure>

A **physical explanation** to this apparent paradox comes when considering that protein and non-coding RNA sequences are **one-dimensional bits of information** but these biomolecules themselves use all **four dimensions** – 3 spatial and 1 temporal – **to define their conformational space**. Yet, as we saw in our protein folding simulations, proteins do not explore the entirety of their possible conformational space (a.k.a. Levinthal’s paradox).

For these biomolecules to efficiently and properly explore their conformational space towards their native structure, **specific (native) contacts between residues must be formed**. In physical terms, these may be hydrogen bonds, salt bridges, Van der Waals interactions, etc. Instead, we can simplify the definition of an interaction between residues by considering an **“interaction”** just as **two residues that have an effect on each other** and are therefore **constrained in the final structure**.

Thus, the adequacy of protein and non-coding RNA sequences to fold into defined three-dimensional structures is not controlled by sequences themselves but controlled by their end product – the conformational space – and **how sequences evolve while keeping these stabilizing interactions between two or more residues in such sequence**.

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/dca_02.gif'/>
<figcaption>Schematic representation of the residue pair contact information in three-dimensional structures that can be extracted from the coevolutionary analysis of protein and RNA sequences. The dotted boxes show how pairs of interacting residues are constrained to coevolve to maintain interactions that are crucial for the stability of the final structure.
Taken from GREMLIN, <i>gremlin.bakerlab.org/gremlin_faq.php</i></figcaption></center>
</figure>

Then, a simple question arises: **can we infer these interactions from sequence information alone?** In the example above, we have a pair of interacting residues that occupy specific positions in a multiple sequence alignment and that can belong to a single or protein chains. As part of random drift, a deleterious substitution may have taken place, partially or completely **disrupting the interaction**, but this could be reversed by a **complementary mutation** that would **restore the interaction** through a different chemistry. This means that for a contact taking place in the native state of a protein, the **residues involved in such interaction can vary in identity** like any other, but they are **mutually restricted to keep identities compatible with such contact**. It ultimately means that the **interactions that are constrained in the final structure are encoded within sequence information as residues that coevolve**.

##Experimental Overview

In this tutorial (which is largely based on the tutorial by Mehari B. Zerihun and deposited on [pyDCA's GitHub](https://github.com/KIT-MBS/pydca)) we will exemplify how we can **infer native contacts from the coevolutionary analysis of sequence information**. For a change, we will work on the coevolutionary analysis of **RNA**, instead of protein, sequences.

We will particularly focus on comparing the experimental and predicted native interactions for the **adenosine deaminase (_add_) A-riboswitch**, one of the structurally simplest riboswitches and member of the Rfam family of **purine riboswitches**. Upon binding to adenine, the aptamer domain of _add_ A-riboswitch changes its folding pattern, which alters the conformation of its downstream expression platform and activates gene expression.

For the coevolutionary analysis of RNA sequences we will employ the **Direct Coupling Analysis (DCA)** method. The DCA scores deliver quantitative information about the existence of physical contacts in a three-dimensional structure of a biomolecule, with top-scoring pairs accurately predicting native contacts that can be observed in experimentally solved protein and RNA structures.

#Part 0. Downloading and Installing the required software

Before we start, **remember to start the hosted runtime** in Google Colab.

Then, we must install several pieces of software to perform this tutorial. Namely:
- **biopython** for manipulation of the PDB files
- **py3Dmol** for visualization of the protein structure.
- **infernal** (INFERence of RNA ALignment) for searching and aligning RNA sequences based on covariance models of sequence and RNA secondary structure consensus
- **pyDCA** for predicting native contacts through direct coupling analysis.

**NOTE:** For the alignment of protein sequences, the recommended option would be **HMMER**, which enables searching and aligning protein sequences based on profile Hidden Markov Model (HMM) of position-specific conservation.

**WARNING⚠️:** pyDCA requires more recent version of several python modules, and thus a **runtime restart** will be requested. Go ahead and accept it!


1. We will first install infernal

In [ ]:
!apt-get install infernal

2. We need to downgrade python to be able to use pyDCA

In [ ]:
!apt-get install python3.7 python3-pip python3.7-distutils -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --config python3

3. Finally, we will install py3Dmol, pydca and biopython through `pip`.
  
  **DO NOT RESTART THE SESSION WHEN ASKED FOR IT, CLICK CANCEL**

In [ ]:
!pip install biopython ipykernel pydca
!python3.10 -m pip install biopython py3Dmol

#Part I – Visualize the structure of _add_ A-riboswitch

We will first download and visualize the solved structure of _add_ A-riboswitch from _Vibrio fulnificus_ (PDB 1Y26) through biopython and py3Dmol.

1. Download the structure of _add_ A-riboswitch as we have done in the past

In [ ]:
#Importing your PDB file using biopython
import os
from Bio.PDB import *
pdbid = ['1y26']
pdbl = PDBList()
for s in pdbid:
  pdbl.retrieve_pdb_file(s, pdir='.', file_format ="pdb", overwrite=True)
  os.rename("pdb"+s+".ent", s+".pdb")

2. Visualize the structure using py3Dmol.

**NOTE:** We are giving you the small exercise of indicating the chain ID for 1Y26 yourself.

In [ ]:
import py3Dmol
#First we assign the py3Dmol.view as view
view=py3Dmol.view()
#The following lines are used to add the addModel class
#to read the PDB files
view.addModel(open('1y26.pdb', 'r').read(),'pdb')
#Here we set the background color as white
view.setBackgroundColor('white')
#Here we set the visualization style and color
view.setStyle({'chain':'X'},{'cartoon': {'colorscheme':'nucleic'}})
#You can activate the labels for each residue if you want
view.addResLabels({'resi':'13-83'},{'fontColor':'black','fontOpacity':1,'showBackground':'false'})
#Here we center the molecule for its visualization
view.zoomTo()
#And we finally visualize the structures using the command below
view.show()

**QUESTION❓:** Are all base-pairing interactions in this riboswitch of Watson-Crick type?

#Part II – Generate a multiple sequence alignment (MSA) of all known purine riboswitch family members using infernal

As previously indicated, the _add_ A-riboswitch is a member of the family of **purine riboswitches** in the [**Rfam** database](http://rfam.xfam.org), a collection of RNA families. The benefit of Rfam is that it contains **seed alignments**, i.e. small MSA based on a subset of known sequences that is used to generate a **consensus covariance model** of sequence and structure conservation.

These models are then used by Rfam to find **all possible RNA homologs for a given family**, and then all of these sequences for each RNA family are also **deposited in this database**.

We will use these seed alignments and sequences of known family members as inputs for generating a covariance model-based MSA using infernal. The resulting MSA will be our input for our coevolutionary analysis, using the sequence of _add_ A-riboswitch as reference for the prediction of residue-pair contacts.

1. We will first download the sequence of 1Y26 in FASTA format using biopython. A small change from previous scripts is that biopython downloads the sequence of interests in DNA alphabet. Thus, in order to obtain our sequence in RNA alphabet, we will create our own record with `SeqRecord` and use `transcribe()` to convert from DNA into RNA.

**NOTE:** We are again giving you the small exercise of letting you set up the correct sequence ID for the RNA we want to download.

In [ ]:
import os
from pathlib import Path
from Bio import SeqIO, Entrez
from Bio.SeqRecord import SeqRecord
seqlist = ['1Y26_X'] # UPPERCASE LETTERS
for n in seqlist:
  #Setting up your email to be able to use Entrez
  Entrez.email = 'your.email@uc.cl'
  #Here, we set up a temporary handle with our downloaded sequence in fasta format
  temp = Entrez.efetch(db="sequences",rettype="fasta",id=n)
  #Creating a fasta file to write our downloaded sequence
  #Which is unfortunately downloaded as DNA
  seq_out = open("1Y26.fasta",'w')
  #Reading the sequence information as a string in fasta format
  seq = SeqIO.read(temp, format="fasta")
  print("Sequence is downloaded as DNA:\n" + seq.seq)
  #Transcribing the sequence, creating a new RNA record  and saving
  rnaseq = seq.seq.transcribe()
  print("We convert it back into RNA:\n"+rnaseq)
  rrec = SeqRecord(rnaseq, id=seq.id, description="reference")
  SeqIO.write(rrec,seq_out,"fasta")
  #Closing both the temp handle and the FASTA file
  temp.close()
  seq_out.close()

2. Then, we will go to the [**Rfam** database](http://rfam.xfam.org) and search for the appropriate RNA family. You can easily do this by inputting _purine_ in the **_Jump To_** search box. This family is annotated as **Purine (RF00167)**.

  Once here, we will obtain two different files:
  - Go to the **_Sequences_** menu to **download all sequences** in FASTA format, which we will use to generate a multiple sequence alignment (MSA)
  - Go to the **_Alignment_** menu and **download the seed alignment**, which is used for constructing a covariance model employed as profile for the MSA. You must download it in Stockholm format from the **_Formatting options_** menu.

  Import both downloaded files by dragging them into the **Files** tab on the left sidebar of Google Colab.

3. We will now use the `cmbuild` module from infernal to generate our covariance model based on the Rfam seed alignment. This model will be stored as a `.cm` file

In [ ]:
!cmbuild RF00167.cm RF00167.stockholm.txt

4. Then, we will align all the downloaded sequences using our readily available covariance model as a MSA profile through the `cmalign` module of infernal. Please note that we are adding our reference sequence into the FASTA file to include it in our MSA. The resulting MSA will be stored as an **aligned FASTA** file named `RF00167_aligned.afa`.

  Given that we are not interested in sequence deletions/insertions relative to the consensus model, we will include match columns in the output alignment with the `--matchonly` option.

In [ ]:
!gunzip RF00167.fa.gz
!cat RF00167.fa 1Y26.fasta > RF00167_full.fasta
!cmalign -o RF00167_aligned.afa --outformat AFA --matchonly RF00167.cm RF00167_full.fasta

We have now obtained a MSA of all the known sequences of the purine riboswitch family (RF00167) of Rfam. As in our previous tutorials, you can visualize this alignment using [Alignment Viewer 2.0](https://fast.alignmentviewer.org/)

#Part III – Coevolutionary analysis of purine riboswitches and accuracy of the prediction of residue pair contacts.

Now, we will use pyDCA, a python implementation of DCA, to first predict residue pairs in contact in the three-dimensional structure of purine riboswitches based on sequence information alone, and then determine the accuracy of our prediction by establishing the number of true positives when compared to the contacts observed in the solved structure of _add_ A-riboswitch.

1. First, we will trim our MSA file based on the lenght of the reference sequence, which in this case corresponds to the sequence of the aptamer of _add_ A-riboswitch that was solved by X-ray crystallography. The starting MSA is defined in the `rna_msa_file` path, whereas our sequence is defined in the `rna_refseq_file`. The trimmed MSA data to an output file `MSA_RF00167_Trimmed.fa` in FASTA format. The program installed is run as follows:

 `pydca trim_by_refseq <biomolecule>  <alignment.fa>  <refseq_file.fa> --remove_all_gaps --verbose`

 This will compare our target sequence to all the sequences in the alignment. Later, it will select the closest one and trim all the gap positions. The resulting msa will be generated in a folder named after the file with the "Trimmed" prefix.

 **NOTE**: To trim protein sequences, we just need to change biomolecule='rna' to biomolecule='protein' and provide the respective MSA and reference sequence files.

In [ ]:
!pydca trim_by_refseq rna RF00167_aligned.afa  1Y26.fasta --remove_all_gaps --verbose

2. We can now check that the top sequence is correctly trimmed

In [ ]:
!head Trimmed_RF00167_aligned/Trimmed_RF00167_aligned.fa

3. Once done, we will compute our DCA scores using a **Pseudolikelihood maximization (plm)** algorithm. This  is different from the **mean-field (mf)** algorithm that we reviewed during the lectures. Analyzing this algorithm is outside the scope of this tutorial, it is elegantly described [in the BiorXiv of pyDCA](https://www.biorxiv.org/content/10.1101/805523v1).

  For this analysis, we first create a plmDCA instance `plmdca_inst` to analyzed our trimmed RNA MSA. We also provide a series of optional parameters. The 2-body couplings $J_{ij}$ and single-site fields $h_i$ and $h_j$ for each position in the MSA are estimated by maximizing their pseudolikelihood from starting values through a gradient descent using an interative approach. The number of iterations is defined by `max_iterations`. Also, a maximum sequence identity percentage of 80% for all sequences in the MSA is established using the `seqid` parameter. Lastly, the number of threads is set to 2, the maximum for Google Colab.

  After the fields and couplings are obtained, the DCA scores are computed from the Direct Information (DI):
<figure>
<center>
<img src='https://github.com/pb3lab/ibm3202/raw/master/images/dca_04.png'/>
</center>
</figure>

  by calling the `compute_sorted_FN_APC()` method on `plmdca_inst`. This action returns the average product corrected (APC) DCA scores.

  Here we will call the `plmdca` function with `compute_fn` function, which will calculate the Frobenius norm of the plmDCA, Average Product Corrected (APC) as we saw in the lectures.


In [ ]:
!plmdca compute_fn rna Trimmed_RF00167_aligned/Trimmed_RF00167_aligned.fa --apc --seqid 0.8 --num_threads 2 --max_iterations 500

4. Let us print our top 10 site pairs and their corresponding DCA scores.

In [ ]:
!head -n 22 PLMDCA_output_Trimmed_RF00167_aligned/PLMDCA_apc_fn_scores_Trimmed_RF00167_aligned.txt

5. Once we have already obtained our DCA scores, we will use the DCA visualizer. This visualizer takes the type of biomolecule (`'rna'`), our reference sequence file (`refseq_file`) and our previously generated list of sorted DCA scores (`sorted_dca_scores`) as input to determine its residue pairs in contact based on sequence information alone.

  When structural information is available, it also takes a PDB ID (`'1y26'`) and chain (`'x'`) as input parameters, as well as specified linear (i.e. sequence separation between residue pairs, `linear_dist`) and contact distances (`contact_dist`, in Å).This visualizer takes the following inputs:



```
pydca plot_contact_map <biomolecule> <PDB_chain_name> <PDB_id/PDB_file.PDB> <refseq.fa> <DCA_file.txt> --verbose  
```

**NOTE:** When we execute the above cell we see a  `You didn't supply RNA secondary structure file` warning. Since supplying an RNA secondary structure file is optional, we can ignore the warning.

In [ ]:
!pydca plot_contact_map rna X 1Y26 1Y26.fasta\
 PLMDCA_output_Trimmed_RF00167_aligned/PLMDCA_apc_fn_scores_Trimmed_RF00167_aligned.txt --verbose

6. We will now plot both contact maps, with the structure-based contacts on the upper left triangle, and the DCA-based contacts on the lower right triangle. Correctly predicted contacts are shown in green, whereas false positives are shown in red. Also note that the number of DCA contacts is equivalent to $L$, where $L$ is the lenght (i.e. number of columns) of the trimmed alignment.

In [ ]:
!awk 'NR>=23{print $5, $6, $1}' /content/contact_map_1Y26/contact_map1Y26.txt > contacts.txt
!echo X Y Z > /content/headers
!cat /content/headers /content/contacts.txt > /content/contacts2.txt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

data = np.genfromtxt('/content/contacts2.txt', dtype=None, names=True, encoding=None)
x = data['X']
y = data['Y']
z = data['Z']

# Define a dictionary to map categories to colors
category_colors = {
    'tp': 'green',
    'fp': 'red',
    'pdb': 'gray'
}

# Create lists to store the inverted coordinates
inverted_x = []
inverted_y = []
colors = []

plt.figure(figsize=(6, 6))

for i in range(len(z)):
    category = z[i]
    if category in ('tp', 'fp'):
        inverted_x.append(y[i])  # Invert X and Y for 'tp' and 'fp' categories
        inverted_y.append(x[i])
        colors.append(category_colors[category])
    else:
        inverted_x.append(x[i])
        inverted_y.append(y[i])
        colors.append(category_colors[category])

plt.scatter(inverted_x, inverted_y, c=colors, marker='o', s=5)  # You can change the 's' value as needed
for category, color in category_colors.items():
    plt.scatter([], [], c=color, label=category)

plt.xlabel('Residue index')
plt.ylabel('Residue index')
plt.title('Contact map for the mfDCA prediction')
plt.legend(bbox_to_anchor=(1.05, 1.05))
plt.grid(True)
plt.show()

7. Lastly, we will determine the accuracy of our coevolutionary analysis by looking at its true positive (TP) rate per rank. The TP rate per rank is the number of correctly predicted contacts per rank of the predicted pairs divided by all predictions at that rank

In [ ]:
!pydca plot_tp_rate rna X 1Y26 1Y26.fasta\
 PLMDCA_output_Trimmed_RF00167_aligned/PLMDCA_apc_fn_scores_Trimmed_RF00167_aligned.txt --verbose

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
data = pd.read_csv('/content/TPR_1Y26/TPR_1Y26.txt', delimiter='\t', skiprows=11, header=None, usecols=[0])

data['Index'] = range(1, len(data) + 1)
data.set_index('Index', inplace=True)
plt.semilogx(data.index, data[0], linestyle='-')
plt.xlabel("Rank")
plt.ylabel("TPR")
plt.title("True positive rate for sorted APC")
plt.show()

In this resulting plot, the blue line corresponds to the TP rates for the predicted contacts, and the orange line corresponds to the theoretically maximum possible true positive rate for the contacts obtained from the PDB structure.

**This is the end of the eleventh tutorial!** Good science!